In [13]:
import os
import time
import pandas as pd
import numpy as np

from konlpy.tag import Okt
from multiprocessing import Pool
#from pororo import Pororo
from konlpy.tag import Kkma

In [5]:
# 멀티 프로세싱을 사용하기 전 쓰레드 확인
os.cpu_count()

2

In [6]:
# 쓰레드 확인 후 프로세스 숫자 결정
process_num = 2

In [14]:
okt = Okt()
kkma = Kkma()

In [20]:
test = okt.pos("시티뷰/리버뷰 모두 가능한 객실 예매했는데 리버뷰는 대기를 좀 오래한다고 하셔서 결국 시티뷰를 선택했는데 조금 아쉬웠습니다. 식탁에 의자가 2개라 밥먹기가 조금 애매했습니다")
test

[('시티', 'Noun'),
 ('뷰', 'Noun'),
 ('/', 'Punctuation'),
 ('리버', 'Noun'),
 ('뷰', 'Noun'),
 ('모두', 'Noun'),
 ('가능한', 'Adjective'),
 ('객실', 'Noun'),
 ('예매', 'Noun'),
 ('했는데', 'Verb'),
 ('리버', 'Noun'),
 ('뷰', 'Noun'),
 ('는', 'Josa'),
 ('대기', 'Noun'),
 ('를', 'Josa'),
 ('좀', 'Noun'),
 ('오래', 'Adverb'),
 ('한다고', 'Verb'),
 ('하셔서', 'Verb'),
 ('결국', 'Adverb'),
 ('시티', 'Noun'),
 ('뷰', 'Noun'),
 ('를', 'Josa'),
 ('선택', 'Noun'),
 ('했는데', 'Verb'),
 ('조금', 'Noun'),
 ('아쉬웠습니다', 'Adjective'),
 ('.', 'Punctuation'),
 ('식탁', 'Noun'),
 ('에', 'Josa'),
 ('의자', 'Noun'),
 ('가', 'Josa'),
 ('2', 'Number'),
 ('개', 'Noun'),
 ('라', 'Josa'),
 ('밥', 'Noun'),
 ('먹기', 'Noun'),
 ('가', 'Josa'),
 ('조금', 'Noun'),
 ('애매했습니다', 'Adjective')]

In [17]:
test2 = okt.pos("고급스러움")
test2

[('고급스러', 'Adjective'), ('움', 'Noun')]

In [8]:
sa = Pororo(task="sentiment", model="brainbert.base.ko.shopping", lang="ko")

In [9]:
hotel_review_df = pd.read_csv('./hotel_review.csv')

In [10]:
# 크롤링 파일 columns 수정 후 삭제
hotel_review_df.rename(columns = {'review_index' : 'review_id', 'hotel_index' : 'hotel_id'}, inplace = True)

In [11]:
hotel_review_df.tail(3)

,review_id,date,review,hotel_id,initial_label
115178,115179,2019-05,고객응대 및 가격대비 방도 넘 안좋아요 난방도 안됨,551,0
115179,115180,2019-03,침대가 너무 딱딱 했어요 이름은 호텔이라고 해 두었지만 모텔 이었어요 모텔 이었다면...,551,0
115180,115181,2020-05,분명 성인 3명 요금으로 2베드 스위트룸으로 예약했는데 체크인하려고 갔더니 카드결제...,551,0


In [12]:
# 테스트용
slice_range = 10000
splited_hotel_review_dfs = [hotel_review_df[(x * slice_range) : ((x+1) * slice_range)] for x in range(1, 11)]
splited_hotel_review_dfs.append(hotel_review_df[110000:])

In [13]:
splited_hotel_review_dfs[-1].tail(3)

,review_id,date,review,hotel_id,initial_label
115178,115179,2019-05,고객응대 및 가격대비 방도 넘 안좋아요 난방도 안됨,551,0
115179,115180,2019-03,침대가 너무 딱딱 했어요 이름은 호텔이라고 해 두었지만 모텔 이었어요 모텔 이었다면...,551,0
115180,115181,2020-05,분명 성인 3명 요금으로 2베드 스위트룸으로 예약했는데 체크인하려고 갔더니 카드결제...,551,0


In [15]:
def set_label(splited_df):
    start = time.time()
    pid = os.getpid()

    copy_splited_df = splited_df.copy()
    error_log = []
    x = ""
    for df in splited_df.itertuples():
        # df.Index의 대소문자 유의
        index = df.Index
        id = df.review_id
        review = df.review

        try:
            sentiment = sa(review)

        # 에러 발생시키는 리뷰 삭제
        except Exception as e:
            error_log.append([id, review, e])
            continue
        
        print(f'pid: {pid}, id: {id}, sentiment: {sentiment}, review: {review}')

        # pororo 결과에 맞춰 label 변경
        copy_splited_df.loc[index, 'initial_label'] = (1 if sentiment == 'Positive' else 0)

    delta_t = round(time.time() - start, 2)
    print('--------------')
    print(f'pid : {pid}')
    print(f'running time : {delta_t}')
    print('--------------')

    if error_log: 
        with open(f'./error_log_{pid}.txt', 'w') as f:
            for error in error_log:
                f.write(str(error)+'\n')

    return copy_splited_df

In [ ]:
# 단일 프로세스로 진행
for index, df in enumerate(splited_hotel_review_dfs):
    labeled_hotel_review = set_label(df)
    labeled_hotel_review.to_csv(f'./{index}_hotel_review.csv', encoding='utf-8')

In [ ]:
'''
slice_range = len(hotel_review_df) // process_num

# 멀티 프로세싱 (colab 사용량 제한으로 데이터가 많아지면 안되는듯, 작은 데이터에서 정상 동작)
with Pool(process_num) as p:
    ret = p.map_async(set_label, [hotel_review_df[(x * slice_range) : ((x+1) * slice_range)] for x in range(process_num)])
    ret_data = ret.get()
'''

In [ ]:
ret_data

In [ ]:
labeled_hotel_review_df = pd.concat(ret_data, axis=0)

In [ ]:
labeled_hotel_review_df

In [ ]:
labeled_hotel_review_df.to_csv('./labeled_hotel_review.csv', encoding='utf-8')

처음 label initialize 할때 사용하려 했던 코드
너무 오래 걸려 Pororo 한번 돌리기로 함
차후 SVM 사용시 재활용 가능할듯

In [ ]:
remove_index_list = []
positive_index_list = []
negative_index_list = []

for review in wrong_review:
  index = review[0]
  print(review)
  print('긍정: P, 부정: N, 삭제: 아무키나 or 빈값')
  input_key = input().upper()

  if input_key == "P":
    positive_index_list.append(index)
  elif input_key == "N":
    negative_index_list.append(index)
  else:
    remove_index_list.append(index)


[6, '화장실 개수구가 고장나서 물이 안빠짐\n객실 안내 전 꼼꼼한 수리가 필요해보임', 1]
긍정: P, 부정: N, 삭제: 아무키나 or 빈값
n
[7, '객실에 의자가 1개 밖에 없어서 불편했어요.\n프론트에 요청을 했지만 남아 있는 의자가 없어서 서비스가 불가능하다는 답변을 받았습니다.', 1]
긍정: P, 부정: N, 삭제: 아무키나 or 빈값
n
[8, '음식 가지수가 많지 않앗어요 \n핸드폰 충전기가 부족해 이용 하지 못햇습니다', 1]
긍정: P, 부정: N, 삭제: 아무키나 or 빈값
n
[9, '새벽부터 아침까지 알수없는 소음으로 잠을 설쳤습니다 ㅠㅠ', 1]
긍정: P, 부정: N, 삭제: 아무키나 or 빈값
n
[10, '아무리 찾아도 어메니티 중에 빗이 없음;;\n깨끗하고 넓은 거외엔 이렇다할 특이사항이 없음.\n두명이 쓰는데 테이블 의자가 하나인것도 아쉬움', 1]
긍정: P, 부정: N, 삭제: 아무키나 or 빈값
n
[11, '욕실안에 있는 컵이 너무 위험해요\n아이가 양치하다가 깨서 변상도했습니다', 1]
긍정: P, 부정: N, 삭제: 아무키나 or 빈값
n
[12, '5성급 호텔이라는 점을 감안하면 조식에서 다소 아쉬웠습니다. 첫방문이라 ZEST 주방인원분들이 평소보다 적었는지는 알 수 없었지만 음식이 리플레이팅 되는 시간이 오래 걸렸습니다.', 1]
긍정: P, 부정: N, 삭제: 아무키나 or 빈값
n
[14, '방음이 너무 안되는듯요. 방바깥에서 사람들 왔다갔다 하는\n소리며 문여닫는 소리가 너무 잘 들려서 방안 소리도 그렇게 들릴거 같아 신겅쓰였어요.\n게다가 주차등록은 두대까지 된다 하시고는 한대만 등록해두셔서 나중에 출차힐때 좀 번거로웠네요.', 1]
긍정: P, 부정: N, 삭제: 아무키나 or 빈값
n
[15, '발렛에 국산차는 지하로 내리는게 룰인가요?ㅋㅋ 기다리는데 이십분 쏟앗습니당!', 1]
긍정: P, 부정: N, 삭제: 아무키나 or 빈값
n
[16, '체크인 시간 한참뒤에 객실배정받음(레이트체크아웃해

KeyboardInterrupt: ignored

In [ ]:
remove_index_list

[91, 94, 138, 172, 176, 423, 471, 484, 488, 503, 524, 536, 580, 652]

In [ ]:
positive_index_list

[59, 104, 121, 418, 463, 491]

In [ ]:
negative_index_list

[6,
 7,
 8,
 9,
 10,
 11,
 12,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 46,
 47,
 48,
 49,
 51,
 52,
 53,
 54,
 55,
 56,
 58,
 60,
 61,
 62,
 63,
 65,
 66,
 67,
 68,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 81,
 82,
 83,
 84,
 86,
 87,
 88,
 89,
 90,
 92,
 93,
 95,
 96,
 97,
 98,
 100,
 101,
 102,
 103,
 105,
 106,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 132,
 133,
 135,
 136,
 137,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 150,
 151,
 152,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 167,
 168,
 169,
 170,
 171,
 173,
 174,
 175,
 177,
 178,
 179,
 180,
 182,
 183,
 184,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 203,
 204,
 205,
 206,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218

In [ ]:
removed_hotel_review_df = hotel_review_df.drop(remove_index_list, axis=0)

In [ ]:
removed_hotel_review_df.loc[positive_index_list,'initial_label'] = 1

In [ ]:
removed_hotel_review_df.loc[negative_index_list, 'initial_label'] = 0

In [ ]:
removed_hotel_review_df.to_csv('./hotel_review_from_0')

In [ ]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 4.6 MB/s 
     |████████████████████████████████| 448 kB 64.5 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import gensim
from konlpy.tag import Okt
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
okt = Okt()

In [ ]:
# 리뷰 데이터 가져오기
tagged_review = pd.read_csv('./data/sample_hotel_review.csv')

In [ ]:
df_cleansing = pd.DataFrame(
        columns=["sentence", "cleansed_target", "cleansed_stopwords"]
    )

In [ ]:
tagged_review.head(3)

,review_index,date,review,hotel_index,label,classified_label
0,0,2021년 6월,조식이 세미 뷔페식이었어요. \n여자친구는 건강식 전 펜케익을 먹었었는데 맛은 매우...,1,1,NaN
1,1,2022년 1월,뷰가 좋아요!,1,1,NaN
2,2,2022년 1월,"우선, 예약 상 문제가 있었던 부분에 대해 콘래드 측에서 즉각적인 대응을 해주셔서,...",1,1,NaN


In [ ]:
def cleansing_wo_komoran(document):
    # okt.tagset => 품사확인

    # document normalization
    norm_doc = okt.normalize(document)
    token_list = okt.pos(norm_doc, join=True)
    targets = ["Adjective", "Adverb", "Noun", "Verb"]

    # targets에 해당되는 token만 추출해 저장
    doc_tokenized = []
    for token in token_list:
        for target in targets:
            if target in token:
                dash_index = token.index("/")
                doc_tokenized.append(token[0:dash_index])

    # stopwords 제거한 token 저장
    stop_words = "이	있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 소리 놓"
    stop_words = set(stop_words.split(" "))
    cleansed_doc = [word for word in doc_tokenized if not word in stop_words]

    return cleansed_doc

In [ ]:
tagged_review = tagged_review.drop([10634], axis=0)

In [ ]:
# review_list를 가져와서 명사만 추출한 후 tag(index)와 함께 tagged_corpus_list에 저장
tagged_corpus_list = []
for i in range(len(tagged_review)):
    if i == 10634: continue
    cleansed_doc = cleansing_wo_komoran(tagged_review.loc[i]['review'])
    try:
        tagged_line = TaggedDocument(tags=[f'{tagged_review.loc[i]["hotel_index"]}@{i}'], words=cleansed_doc)
        tagged_corpus_list.append(tagged_line)
    except Exception as e:
        print('Error in tagged_corpus')
        continue
    print(f'---{round((i / len(tagged_review)) * 100 ,5)}%---')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
---68.91514%---
---68.92136%---
---68.92757%---
---68.93379%---
---68.94001%---
---68.94622%---
---68.95244%---
---68.95866%---
---68.96487%---
---68.97109%---
---68.97731%---
---68.98353%---
---68.98974%---
---68.99596%---
---69.00218%---
---69.00839%---
---69.01461%---
---69.02083%---
---69.02704%---
---69.03326%---
---69.03948%---
---69.04569%---
---69.05191%---
---69.05813%---
---69.06435%---
---69.07056%---
---69.07678%---
---69.083%---
---69.08921%---
---69.09543%---
---69.10165%---
---69.10786%---
---69.11408%---
---69.1203%---
---69.12652%---
---69.13273%---
---69.13895%---
---69.14517%---
---69.15138%---
---69.1576%---
---69.16382%---
---69.17003%---
---69.17625%---
---69.18247%---
---69.18869%---
---69.1949%---
---69.20112%---
---69.20734%---
---69.21355%---
---69.21977%---
---69.22599%---
---69.2322%---
---69.23842%---
---69.24464%---
---69.25085%---
---69.25707%---
---69.26329%---
---69.26951%---
---69.27572%---
---69.28194%---
---69.2881

## 토큰 길이가 긴 리뷰 확인

In [ ]:
for word, tag in tagged_corpus_list:
    if len(word) >= 200:
        print(len(word), tag, word)
        #sa(tagged_review.loc[int(tag[0].split('@')[-1])]['review'])

260 ['1@360'] ['호텔', '스위트룸', '샤워', '기', '해바라기', '수전', '망가져서', '물이', '사방', '튐', '샤워실', '바깥', '물이', '튀고', '선반', '젖음', '대신', '욕조', '이용', '하려고', '보니', '욕조', '배', '수구', '머리카락', '끼', '빠지지도', '않음', '룸체', '요청', '하자', '연락', '준다고', '처리', '해달라', '아니고', '직원', '언급', '확인', '할', '직원', '보내줄테니', '빨리', '짐', '싸놓고', '기다리라', '더니', '지나도', '연락', '없어', '계속', '먼저', '확인', '전화', '하게', '만듦', '연락', '할', '마지막', '단계', '끝났다며', '조금', '기다리라는데', '결국', '방', '교체', '걸리고', '밤', '씻지도', '자지', '못', '하게', '함', '젖은', '수영복', '짐', '함께', '가방', '들어가서', '가방', '축축해', '짐', '차라리', '죄송하지만', '언제', '걸릴', '같으니', '양해', '해달라', '고', '했다면', '망가진', '수전', '쪽', '대충', '씻고', '짐', '빼놓고', '했을텐데', '먼저', '물어볼', '곧', '직원', '갈거다', '준비', '끝나서', '곧', '된다', '하며', '이러지도', '저러지도', '못', '하게', '함', '호텔', '먼저', '해놓고', '지키지도', '않으며', '연락', '주지', '않고', '오매불망', '듯이', '기다리게', '만들고', '먼저', '연락', '하면', '금방', '되니', '기다리라는', '헛소리', '함', '수영장', '다녀온', '후', '더럽게', '찝찝하고', '어린아이까지', '있어서', '그라데이션', '분노', '차오름', '시설', '따위', '서비스', '받으려고', '돈', '주고', '여길', '와서', '취급', '받나', '

In [ ]:
review = tagged_review.loc[843]['review']

In [ ]:
review

'대체 뭐하는거지 싶었던 직원. 예약이 되었냐니까 그냥 자기들은 모르겠으니 숙박중개업체에 가서 이야기하라고 하는데 내가 예약한 시간은 저녁 8시인가 그랬음. 1박에 60-70만원을 쓰는데 내가 무슨 뷰를 보는지도 모르고 숙소까지 갔음 .. 모른다, 해줄 수 없다, 알 수 없다, 그건 안된다라는 말만 게~속 하는 상담직원이 대체 무슨 의미가 있나 싶고, 세상에 방을 예약했는데 그 방이 어떤 뷰인지 모르는 직원이 있다는 것은 더 놀라움. 도착하고 나니까, 숙박 예약담당직원이 퇴근해서 자기는 담당자가 아니라고 다시 확인해봐야된다고함. 8시에 예약해서 11시인가? 12시인가? 체크인한거로 기억하는데 ㅋㅋ 그럼 3시간동안 대체 무슨 일이 ..? 결국 우여곡절끝에 결제한다고 했더니, 법인카드로는 내가 그 법인의 관계자인지 아닌지 알 수가 없기 때문에 개인카드로 결제하거나 현금으로 결제하거나 계좌이체를 하라고함 .. 예약 당시에 데스크에서 금액인포 해주지 말라고 얘기했는데 그런것도 없이 정가 얼마얼마입니다 ^^ 입금바랍니다. 하는데 표정관리가 도저히 안됬음. 이거로 컴플레인을 거니까 자기는 이 호텔의 매니저고, 내가 책임자인데 난 그런거 들은거 없고 어쩌라고? 라는 반응. 호텔만 좋지, 진짜 서비스는 쓰레기구나 라는 생각이 들었고, 차지포함 100만원 계좌이체해주고 입실 ㅋㅋㅋ 들어가는데, 고객카드 주는거에는 내 이름이 아니라 딴 사람 이름이 써있음. 매니저님 진짜 너무하신거 아닙니까? 하더니 죄송하다고함. 그냥 됬고 빨리 들어가서 쉬고 싶다니까 아닙니다 고쳐야됩니다 하더니 자기가 새로 뽑아서 줌 (한 체감상 5-10분? 정도 이거로 지연) 대체 무슨 기준으로 서비스를 교육하고 ,무슨 기준으로 서비스를 제공하는건지 알 수가 없는 호텔이였음. 리뷰 찾아보니 나만 이런거 느낀 것도 아니고 호텔은 좋은데, 서비스는 노답이라는 평이 괜히 있는게 아니구나 .. 라는 생각. 그럼 들어가서는 좋았냐? 리버뷰로 북쪽이 보이는 방이라고 안내받아서 들어갔고, 방 보자마자 와 .. 이래서 오

In [ ]:
len(tagged_corpus_list[843][0])

337

In [ ]:
sa(tagged_review.loc[2552]['review'])

ValueError: ignored